In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
import os

# ブランドフォルダのリスト
# brands = ['uniqlo', 'gunze', 'muji']
brands = ['gunze', 'muji']

# 各ブランドフォルダ内のcsvファイルを取得し、データフレームのリストに追加
dataframes = []
for brand in brands:
    csv_files = [f for f in os.listdir(f'../data/{brand}') if f.endswith('.csv')]
    for f in csv_files:
        df = pd.read_csv(os.path.join(f'../data/{brand}', f))
        df['brand'] = brand  # brand列を追加
        dataframes.append(df)

# データフレームを縦に結合
merged_df = pd.concat(dataframes, ignore_index=True)

# review列を1文ごとに分割し、それぞれの文ごとに行を作成
split_reviews = merged_df['review'].str.split('[。!?！？]', expand=True).stack().reset_index(level=1, drop=True)
split_reviews.name = 'review'

# 分割されたレビューを元に新しいデータフレームを作成
expanded_df = merged_df.drop(columns=['review']).join(split_reviews)

# review列の欠損値(空白文字)を削除
expanded_df = expanded_df[expanded_df['review'] != '']

# review_id列を作成し、そのときのindexを格納
expanded_df['review_id'] = expanded_df.index

expanded_df

,product,star,gender,age,date,brand,review,review_id
0,半袖丸首,4,男性,NaN,2024-09-21,gunze,快適工房1択の夫にプレゼントしました,0
0,半袖丸首,4,男性,NaN,2024-09-21,gunze,肌着として着用していて、首元が見えてしまっていてもグレーなので、下着という感じがしなくて良い...,0
1,半袖丸首,3,男性,NaN,2024-07-22,gunze,着た直後の着心地はサラッとして良いです,1
1,半袖丸首,3,男性,NaN,2024-07-22,gunze,購入したのはLサイズですが、体にピタッとフィットします,1
1,半袖丸首,3,男性,NaN,2024-07-22,gunze,暑い日にはしっかり汗を吸ってくれますので、ボディ周りの汗染みは防いでくれます,1
...,...,...,...,...,...,...,...,...
1130,あったか綿 クルーネック長袖Ｔシャツ（２０２３年仕様）,5,男性,/40代,2023/11/06,muji,\n首元も寒くなくていいです,1130
1131,あったか綿 クルーネック長袖Ｔシャツ（２０２３年仕様）,4,NaN,NaN,2023/10/21,muji,思ったより薄手です,1131
1131,あったか綿 クルーネック長袖Ｔシャツ（２０２３年仕様）,4,NaN,NaN,2023/10/21,muji,引っ張るとちょっと透け感があるような・・,1131
1131,あったか綿 クルーネック長袖Ｔシャツ（２０２３年仕様）,4,NaN,NaN,2023/10/21,muji,1枚では着れないです,1131


In [33]:
# expanded_dfのbrand列の要素数の割合を基に1000件のデータをサンプリング(層化抽出)
# 各ブランドの割合を計算
brand_counts = expanded_df['brand'].value_counts(normalize=True)

# 各ブランドからサンプリングする数を計算
sample_sizes = (brand_counts * 1000).round().astype(int)

# 各ブランドからサンプリング
sampled_df = expanded_df.groupby('brand').apply(lambda x: x.sample(sample_sizes[x.name], random_state=42)).reset_index(drop=True)

# サンプリングされたデータフレームを確認
sampled_df

/var/folders/93/1mq4j7t110j7w75f3f3wk4kw0000gn/T/ipykernel_32078/1220507939.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = expanded_df.groupby('brand').apply(lambda x: x.sample(sample_sizes[x.name], random_state=42)).reset_index(drop=True)


,product,star,gender,age,date,brand,review,review_id
0,ＵネックＴシャツ(深めのUネック)(Ｕ首),5,男性,NaN,2023-11-26,gunze,温かく過ごせるので助かります,12
1,【綿混起毛】8分袖ウォーマー（パッド付）,5,女性,NaN,2024-02-17,gunze,大病してブラジャーがつけられなくなったので、この商品はとてもありがたいです,86
2,【綿混起毛】8分袖ウォーマー（パッド付）,5,女性,NaN,2023-01-19,gunze,他の下着が着れなくなりました,131
3,【綿混起毛】8分袖ウォーマー（パッド付）,5,女性,NaN,2022-11-23,gunze,冬には欠かせないアイテムになりました,145
4,【綿混起毛】8分袖ウォーマー（パッド付）,5,女性,NaN,2024-03-07,gunze,冬は上着も着ているので、パットでもシルエットも気にならないし、夏以外はずっときています,82
...,...,...,...,...,...,...,...,...
995,あったか綿厚手Ｕネック八分袖Ｔシャツ（２０２３年仕様）,1,女性,NaN,2024/02/18,muji,」となりました\n残念です,1035
996,婦人さらっと綿汗取りパッド付きフレンチスリーブＴシャツ,2,女性,/40代,2024/07/29,muji,それがすごい良かったので今年も同じ物と思い何も考えずに手に取りました,749
997,婦人さらっと綿汗取りパッド付きタンクトップ,5,女性,/40代,2024/06/07,muji,今まで色々な物を試しましたがこれは良いです,979
998,婦人さらっと綿汗取りパッド付きフレンチスリーブＴシャツ,4,女性,/50代,2024/08/03,muji,サイズてきにはピッタリフィットして、脇の部分も汗を吸い取りやすいです,745


In [34]:
# merged_dfからreview_id, review列のみ取り出す
review_df = sampled_df[['review_id', 'review']]
review_df['suggest'] = np.nan

# 一度ファイルオブジェクトをエラー無視して、書き込みで開くようにする
# with open("../data/review_3.csv", mode="w", encoding="shift-jis", errors="ignore") as f:
#     # ここでデータフレームを開いたファイルにcsvで書き込む
#     review_df.to_csv(f)

/var/folders/93/1mq4j7t110j7w75f3f3wk4kw0000gn/T/ipykernel_32078/1675292180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df['suggest'] = np.nan


In [44]:
review_uni_df = pd.read_csv("../data/review_uniqlo.csv")
# review_dfに列名を揃えたい
review_uni_df = review_uni_df[['口コミ', 'suggest']]
review_uni_df.columns = ['review', 'suggest']

# review列のnanaを削除
review_uni_df = review_uni_df.dropna(subset=['review'])

# # suggest列をint型に変換
# review_uni_df['suggest'] = review_uni_df['suggest'].astype(int)


# review_idという列を作成し、1132から始まる数値を降ってください。
review_uni_df['review_id'] = range(1132, 1132 + len(review_uni_df))

review_uni_df

,review,suggest,review_id
0,生地も柔らかく着心地は良いですね,0.0,1132
1,ヒートテックの暖かさはは言わずもがなですが、下に1枚着るだけで暖かさが保たれます,0.0,1133
2,下着のヒートテックだとＴシャツと重ねるのはあまり好きではないのですが、この商品なら1枚でサラ...,0.0,1134
3,\nお値段的にも気軽に購入でき、冬の定番になりそうです,0.0,1135
5,少しぴったり目が好きなので、満足している,0.0,1136
...,...,...,...
2239,\nただ、もう少し暖かいものをと超極暖にしたいのですが、Uネックがありません,0.0,2874
2240,\n女性の服は首回りが広いものも多いです,0.0,2875
2241,\nユニクロさん作ってくださいお願いします,1.0,2876
2243,今までのヒートテックだと痒くなってしまっていたのですが、肌に触る方が綿なので、全く問題ありません,0.0,2877


In [46]:
review_2_df = pd.read_csv("../data/review_2.csv", encoding="shift-jis")
review_2_df

,Unnamed: 0,review_id,review,suggest
0,0,12,温かく過ごせるので助かります,0.0
1,1,86,大病してブラジャーがつけられなくなったので、この商品はとてもありがたいです,0.0
2,2,131,他の下着が着れなくなりました,0.0
3,3,145,冬には欠かせないアイテムになりました,0.0
4,4,82,冬は上着も着ているので、パットでもシルエットも気にならないし、夏以外はずっときています,0.0
...,...,...,...,...
995,995,1035,」となりました\n残念です,0.0
996,996,749,それがすごい良かったので今年も同じ物と思い何も考えずに手に取りました,0.0
997,997,979,今まで色々な物を試しましたがこれは良いです,0.0
998,998,745,サイズてきにはピッタリフィットして、脇の部分も汗を吸い取りやすいです,0.0


In [48]:
# review_uni_dfとreview_2_dfを縦に結合
review_new_df = pd.concat([review_uni_df, review_2_df], ignore_index=True)
# Unnnamed: 0列を削除
review_new_df = review_new_df.drop(columns=['Unnamed: 0'])

review_new_df

,review,suggest,review_id
0,生地も柔らかく着心地は良いですね,0.0,1132
1,ヒートテックの暖かさはは言わずもがなですが、下に1枚着るだけで暖かさが保たれます,0.0,1133
2,下着のヒートテックだとＴシャツと重ねるのはあまり好きではないのですが、この商品なら1枚でサラ...,0.0,1134
3,\nお値段的にも気軽に購入でき、冬の定番になりそうです,0.0,1135
4,少しぴったり目が好きなので、満足している,0.0,1136
...,...,...,...
2742,」となりました\n残念です,0.0,1035
2743,それがすごい良かったので今年も同じ物と思い何も考えずに手に取りました,0.0,749
2744,今まで色々な物を試しましたがこれは良いです,0.0,979
2745,サイズてきにはピッタリフィットして、脇の部分も汗を吸い取りやすいです,0.0,745


In [57]:
# review_new_dfから2100件のデータをサンプリング
# review_new_df = review_new_df.sample(2100, random_state=42)
# review_new_df.to_csv("../data/review_new.csv", index=False)

# 一度ファイルオブジェクトをエラー無視して、書き込みで開くようにする
# with open("../data/review_new_shiftjis.csv", mode="w", encoding="shift-jis", errors="ignore") as f:
#     # ここでデータフレームを開いたファイルにcsvで書き込む
#     review_new_df.to_csv(f)

In [60]:
review_new_df = pd.read_csv("../data/review_new_shiftjis.csv", encoding="shift-jis")
# review_new_df.to_csv("../data/review_new.csv", index=False)

In [66]:
review_suggest_df = review_new_df[review_new_df['suggest'] == 1]
review_suggest_df.to_csv("../data/review_suggest.csv", index=False)

review_suggest_df

,Unnamed: 0,review,suggest,review_id
7,565,オフホワイトじゃ無くて白だったらもっと良かったです！,1.0,1697
12,782,あるので、もう少し首元が広めのも作って頂きたいです,1.0,1914
18,239,以前にlLを購入したら少し大きめでしたので今回はMを購入ジャストフィット出来れば今年の冬には...,1.0,1371
20,1017,首回りもっと広くして下さい,1.0,2149
28,840,ただ、首回りが詰まってるので、私的にはもう少し首回りが広い種類もあると嬉しいです,1.0,1972
...,...,...,...,...
2067,280,サイズも暖かさも全く問題ないけど、長袖があったらよかった,1.0,1412
2077,1473,ただ、首が締め付けられる感じがあるので、いつもワンサイズ大きいものを買うため、首にもう少し緩...,1.0,2605
2082,796,だったらメンズ買えば？と思うけど、着丈が長くなるのでレディースサイズでほしいです,1.0,1928
2088,248,極暖にもタートルネックが欲しい,1.0,1380


In [72]:
reviewer_1 = pd.read_csv("../data/review_suggest.csv")
reviewer_1 = reviewer_1.drop(columns=['Unnamed: 0'])

# review_id列で昇順にソート
reviewer_1 = reviewer_1.sort_values('review_id')

reviewer_1["suggest"] = np.nan

# 列の順番を入れ替え (review_id, review, suggest)
reviewer_1 = reviewer_1[['review_id', 'review', 'suggest']]
reviewer_1.to_csv("../data/reviewer_1.csv", index=False, encoding="shift-jis")
reviewer_1

,review_id,review,suggest
109,25,カッティングは綺麗なので、見えてもいいように、縫い目を全部シームレスにして、ストラップも細め...,NaN
98,150,来年は袖10分丈ができたらいいなぁ,NaN
214,163,是非、胸当てやや下付きの商品も出してほしいと思いました,NaN
200,251,大手通販サイトのAやRを参考にして感覚的に何を購入したか一目で写真みたらわかるデザインにし...,NaN
170,292,できれば紺色など他の色も追加してほしいです,NaN
...,...,...,...
186,2856,Vネックカーディガンを1枚で着たいのですがUネックでは見えてしまうので来年は是非Vネックを...,NaN
220,2862,\n去年までの物が欲しいー！,NaN
157,2864,八分袖という丈ですが、わたしには長袖になってしまうので、七分袖、五分袖丈など、もう少し短い...,NaN
27,2870,サイズも小さくなったような… 改善されることを期待しています,NaN


In [73]:
# 上位100件のデータをreviewer_2データとして抽出
reviewer_2 = reviewer_1.head(100)
reviewer_2.to_csv("../data/reviewer_2.csv", index=False, encoding="shift-jis")
reviewer_2


,review_id,review,suggest
109,25,カッティングは綺麗なので、見えてもいいように、縫い目を全部シームレスにして、ストラップも細め...,NaN
98,150,来年は袖10分丈ができたらいいなぁ,NaN
214,163,是非、胸当てやや下付きの商品も出してほしいと思いました,NaN
200,251,大手通販サイトのAやRを参考にして感覚的に何を購入したか一目で写真みたらわかるデザインにし...,NaN
170,292,できれば紺色など他の色も追加してほしいです,NaN
...,...,...,...
43,1554,個人的には普通丈でいいと思います,NaN
139,1557,希望ですが、昔の様に赤やネイビーが有ると有難いです,NaN
59,1567,もっと色味があれば嬉しい,NaN
187,1573,出来れば色のバリエーションを多くして欲しいです,NaN


In [62]:
review_new_df

,Unnamed: 0,review,suggest,review_id
0,2678,洗濯→乾燥→縮む,0.0,882
1,2713,\n買い足そうと思います,0.0,1126
2,2261,温かいとは思います,0.0,1069
3,2519,今のところ都内の12月初旬はコートの下にこれ一枚で暑いくらいです,0.0,1046
4,1561,ヒートテックは以前から愛用していますが、裏地コットンは暖かい上に肌あたりが良くて、ますます使...,0.0,2693
...,...,...,...,...
2095,434,ピタっと肌に密着しているので暖かい,0.0,1566
2096,577,父が外で作業するとき用に購入しました,0.0,1709
2097,2186,締め付け感もなくて楽に過ごせます,0.0,37
2098,2514,一度、試しに着てみて肌触りも良いので夏に向けて4枚購入しました,0.0,801


In [ ]:
r

In [64]:
review_new_df['suggest'].value_counts()

suggest
0.0    1831
1.0     268
Name: count, dtype: int64

In [13]:
# import os

# # data/mujiフォルダ内のすべてのcsvファイルを取得
# csv_files = [f for f in os.listdir('../data/gunze') if f.endswith('.csv')]

# # 各csvファイルを読み込み、データフレームのリストに追加
# dataframes = [pd.read_csv(os.path.join('../data/gunze', f)) for f in csv_files]

# # データフレームを縦に結合
# merged_df = pd.concat(dataframes, ignore_index=True)